In [ ]:
import numpy as np

# image_list = []
text_list = {}
word_list = {}
for image_name in images_hdf5.keys():
    image = images_hdf5[image_name][...]
    text_list[image_name] = images_hdf5[image_name].attrs['txt']
    word_list[image_name] = images_hdf5[image_name].attrs['wordBB']
    jpeg_image = Image.fromarray(image, 'RGB')
    image_list.append(jpeg_image)
    jpeg_image.save('./JPEGImages/' + image_name + '.jpg')

root_hdf5.close()

In [ ]:
# def plot_bounding_box(image_id, box_list):
#     image = np.array(image_list[image_id])
#     boxes = np.array(word_list[image_id])  
#     plt.close(1)
#     plt.figure(1, figsize = (15, 15))
#     plt.imshow(image)
#     H, W = image.shape[:2]
    
#     for index in box_list:
#         bb = boxes[:,:,index]
#         x_min = min(bb[0,:])
#         x_max = max(bb[0,:])
#         y_min = min(bb[1,:])
#         y_max = max(bb[1,:])
#         plt.plot([x_min, x_min, x_max, x_max, x_min], [y_min, y_max, y_max, y_min, y_min], 'r')
        
#     plt.gca().set_xlim([0,W-1])
#     plt.gca().set_ylim([H-1,0])
#     plt.show(block=False)

In [ ]:
# plot_bounding_box(5,[10] )

In [ ]:
# text_list[5]

In [1]:
import xml.etree.cElementTree as ET
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
import os
import shutil
import random
import pickle
from PIL import Image

In [2]:
directory='./Refined_Dataset/'
dataset='./out_fifth'
val=0.10
test=0.15

In [3]:
validClass=['acrylic', 'cotton', 'linen', 'nylon', 'polyester', 'rayon', 'wool']

In [4]:
test=val+test

In [5]:
def check_and_mkdir(in_dir):
    if not os.path.exists(in_dir):
        print("Creating "+in_dir+"...")
        os.makedirs(in_dir)

In [6]:
os.system('rm -rf '+directory)
check_and_mkdir(directory)
check_and_mkdir(directory+'Annotations')
check_and_mkdir(directory+'ImageSets')
check_and_mkdir(directory+'JPEGImages')
check_and_mkdir(directory+'ImageSets'+os.sep+"Main")

Creating ./Refined_Dataset/...
Creating ./Refined_Dataset/Annotations...
Creating ./Refined_Dataset/ImageSets...
Creating ./Refined_Dataset/JPEGImages...
Creating ./Refined_Dataset/ImageSets/Main...


In [7]:
f_train=open(directory+"ImageSets"+os.sep+"Main"+os.sep+"train.txt",'w+')
f_val=open(directory+"ImageSets"+os.sep+"Main"+os.sep+"val.txt",'w+')
f_test=open(directory+"ImageSets"+os.sep+"Main"+os.sep+"test.txt",'w+')
f_trainval=open(directory+"ImageSets"+os.sep+"Main"+os.sep+"trainval.txt",'w+')

In [ ]:
def getXml(image_name,widthh,heightt):
#     print(annotation_path)
#     print(annotation_path.split('.jpg'))
    
#     annotation_path=annotation_path.split('.jpg')[0]+".json"
#     if(not os.path.isfile(annotation_path)):
#         return None
#     jsonObj=json.load(open(annotation_path))

    root = ET.Element("annotation")
    folder = ET.SubElement(root, "folder").text="0"
    filename = ET.SubElement(root, "filename").text="0"
    path = ET.SubElement(root, "path").text="0"
    source = ET.SubElement(root, "source")
    database = ET.SubElement(source, "database").text="Unknown"
    size = ET.SubElement(root, "size")
    width = ET.SubElement(size, "width").text=str(widthh)
    height = ET.SubElement(size, "height").text=str(heightt)
    depth = ET.SubElement(size, "depth").text='3'
    segmented = ET.SubElement(root, "segmented").text='0'
    bounding_boxes = pickle.load(open(os.path.join(dataset + '_bb_out', image_name + '_bb.p') , 'rb'))
    text_annotations = pickle.load(open(os.path.join(dataset + '_bb_out', image_name + '_words.p'), 'rb'))
    objs=len(bounding_boxes)
    actualObj=0
    
    for i in range(objs):
        object_name = text_annotations[i].lower()
        bounding_box = bounding_boxes[i]
        if(object_name not in validClass):
            continue
        actualObj+=1
        object = ET.SubElement(root, "object")
        name = ET.SubElement(object, "name").text=object_name
        pose = ET.SubElement(object, "pose").text='Unspecified'
        truncated = ET.SubElement(object, "truncated").text='0'
        difficult = ET.SubElement(object, "difficult").text='0'
        bbox = ET.SubElement(object, "bndbox")
        xmin = ET.SubElement(bbox, "xmin").text=str(max(bounding_box[0],1))
        ymin = ET.SubElement(bbox, "ymin").text=str(max(bounding_box[1],1))
        xmax = ET.SubElement(bbox, "xmax").text=str(min(bounding_box[2],widthh-1))
        ymax = ET.SubElement(bbox, "ymax").text=str(min(bounding_box[3],heightt-1))

    if(actualObj==0):
        return None
    tree = ET.ElementTree(root)
    return tree
    

In [ ]:
def get_bounding_box(image_name):
    global word_list
    boxes = np.array(word_list[image_name])
    num_boxes = boxes.shape[-1]
    bb_list = []

    for index in range(num_boxes):
        bb = boxes[:,:,index]
        x_min = min(bb[0,:])
        x_max = max(bb[0,:])
        y_min = min(bb[1,:])
        y_max = max(bb[1,:])
        bb_list.append([x_min, y_min, x_max, y_max])
    
    return bb_list

In [ ]:
counter=9466
counter2=0
val_arr=[]
train_arr=[]
test_arr=[]
trainval_arr=[]
for subdir, dirs, files in os.walk(dataset):
    for f in files:
        counter+=1
        print(counter)
        image_path=subdir+os.sep+f
        annotation_path=image_path.lower()
        im = Image.open(image_path)
        if(im is None):
            continue
        width, height = im.size
        tree=getXml(f[:-4],width,height)
        if(tree is None):
            continue
        counter2+=1
        file_counter_name = ('000000' + str(counter))[-6:]
        shutil.copy2(image_path,directory+"JPEGImages"+os.sep+file_counter_name+".jpg")
        tree.write(directory+"Annotations"+os.sep+os.sep+file_counter_name+".xml")
        random_num=random.uniform(0, 1)
        if(random_num<=val):
            val_arr.append(file_counter_name)
            trainval_arr.append(file_counter_name)
        elif(random_num<=test):
            test_arr.append(file_counter_name)
        else:
            train_arr.append(file_counter_name)
            trainval_arr.append(file_counter_name)
#         break
print(counter2)

In [9]:
def printList(lis,filehandler):
    for item in lis:
        print(item, file=filehandler)

In [10]:
printList(val_arr,f_val)
printList(train_arr,f_train)
printList(test_arr,f_test)
printList(trainval_arr,f_trainval)

In [11]:
f_test.close()
f_val.close()
f_train.close()
f_trainval.close()

In [8]:
val_arr=[]
train_arr=[]
test_arr=[]
trainval_arr=[]
for i in range(1, 11467):
    random_num=random.uniform(0, 1)
    file_counter_name = ('000000' + str(i))[-6:]
    if(random_num<=val):
        val_arr.append(file_counter_name)
        trainval_arr.append(file_counter_name)
    elif(random_num<=test):
        test_arr.append(file_counter_name)
    else:
        train_arr.append(file_counter_name)
        trainval_arr.append(file_counter_name)